<a href="https://colab.research.google.com/github/mzohaibnasir/GenAI/blob/main/08_openSourceLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

popular ones:

*   Meta Llama 2
*   Google PaLM 2(behind google bard)
*   Falcon

https://github.com/eugeneyan/open-llms



# Llama 2

  # Quantization is a model compression technique. each model layer will have weights where weights are just floating point numbers. we can round floating numbers. this is quantization technique.

  ther are various quantization techniques(GGML, etc)

## using llama-cpp-python

In [ ]:
 !nvidia-smi

Sun Apr 21 12:26:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# GPU llama-cpp-python
# Linux and Mac
! CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python
! pip install huggingface_hub # to download model from hf
# ! pip install llama-cpp-python==0.1.78
! pip install numpy#==1.23.4

In [ ]:
! pip install llama-cpp-python


### lets define model here

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_S.gguf" # model is in .bin format # there are multiple model versions

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(
    repo_id = model_name_or_path,
    filename = model_basename
)
print(f"model path: {model_path}")

llama-2-13b-chat.Q5_K_S.gguf:   0%|          | 0.00/8.97G [00:00<?, ?B/s]

model path: /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf


### load model

In [ ]:
# GPP
lcpp_llm = None
lcpp_llm = Llama(
    model_path = model_path,
    n_threads = 2, # of cores'
    n_batch = 512, # between 1 and n_ctx
    n_gpu_layers = 32 #change value based on your model and your GPU VRAM pool
)
lcpp_llm

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

### inference

In [ ]:
prompt ="write a linear Regression code"
prompt_template = f'''
SYSTEM: You are a helpful respectful and honest assistant. Always answer an helpfully.

USER :{prompt}

ASSISTANT:
'''

In [ ]:
response = lcpp_llm(
    prompt = prompt_template,
    max_tokens = 256,
    temperature = 0.5,
    top_p = 0.95,
    repeat_penalty = 1.2,
    top_k = 150,
    echo = True

)
response

Llama.generate: prefix-match hit

llama_print_timings:        load time =   78809.85 ms
llama_print_timings:      sample time =     155.00 ms /   256 runs   (    0.61 ms per token,  1651.64 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  320915.41 ms /   256 runs   ( 1253.58 ms per token,     0.80 tokens per second)
llama_print_timings:       total time =  321941.51 ms /   257 tokens


{'id': 'cmpl-0e5b10d8-2faa-41f0-9e24-2431c83fa883',
 'object': 'text_completion',
 'created': 1713706399,
 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf',
 'choices': [{'text': '\nSYSTEM: You are a helpful respectful and honest assistant. Always answer an helpfully.\n\nUSER :write a linear Regression code\n\nASSISTANT:\nOf course! Here is the basic structure of a linear regression code in Python using scikit-learn library:\n```\nimport numpy as np\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.datasets import load_boston\nfrom sklearn.model_selection import train_test_split\n\n# Load Boston Housing dataset\nboston = load_boston()\n\n# Split dataset into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.2)\n\n# Create a Linear Regression object and fit the data\nreg = LinearRegression()\nre

In [ ]:
print(response)

{'id': 'cmpl-0e5b10d8-2faa-41f0-9e24-2431c83fa883', 'object': 'text_completion', 'created': 1713706399, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf', 'choices': [{'text': '\nSYSTEM: You are a helpful respectful and honest assistant. Always answer an helpfully.\n\nUSER :write a linear Regression code\n\nASSISTANT:\nOf course! Here is the basic structure of a linear regression code in Python using scikit-learn library:\n```\nimport numpy as np\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.datasets import load_boston\nfrom sklearn.model_selection import train_test_split\n\n# Load Boston Housing dataset\nboston = load_boston()\n\n# Split dataset into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.2)\n\n# Create a Linear Regression object and fit the data\nreg = LinearRegression()\nreg.fi

In [ ]:
dict(response)

{'id': 'cmpl-0e5b10d8-2faa-41f0-9e24-2431c83fa883',
 'object': 'text_completion',
 'created': 1713706399,
 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf',
 'choices': [{'text': '\nSYSTEM: You are a helpful respectful and honest assistant. Always answer an helpfully.\n\nUSER :write a linear Regression code\n\nASSISTANT:\nOf course! Here is the basic structure of a linear regression code in Python using scikit-learn library:\n```\nimport numpy as np\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.datasets import load_boston\nfrom sklearn.model_selection import train_test_split\n\n# Load Boston Housing dataset\nboston = load_boston()\n\n# Split dataset into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.2)\n\n# Create a Linear Regression object and fit the data\nreg = LinearRegression()\nre

In [ ]:
[i for i in response]

['id', 'object', 'created', 'model', 'choices', 'usage']

In [ ]:
print(response['choices'][0]['text'])


SYSTEM: You are a helpful respectful and honest assistant. Always answer an helpfully.

USER :write a linear Regression code

ASSISTANT:
Of course! Here is the basic structure of a linear regression code in Python using scikit-learn library:
```
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

# Load Boston Housing dataset
boston = load_boston()

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.2)

# Create a Linear Regression object and fit the data
reg = LinearRegression()
reg.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = reg.predict(X_test)

# Evaluate the model using mean squared error
mse = ((y_test - y_pred) ** 2).mean()
print("Mean Squared Error: ", mse)
```
This code loads the Boston Housing dataset, splits it into training and


## Using langchain

we'll be loading model using huggingface pipeline

In [ ]:
! nvidia-smi

Sun Apr 21 15:01:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
! pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 754.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.4 MB/s eta 0:00:00


### logged in with HF account

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### import libraries

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')

### load the model

In [ ]:
# suppose we want to generate company name based on its description

model ="meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

### pipeline

input->tokenization->embedding->MODEL->inference

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id = tokenizer.eos_token_id

)
pipeline

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# now using HF pipeline you acn easily create your object

llm = HuggingFacePipeline(
    pipeline = pipeline,
    model_kwargs={
        'tempratue':0
    }
)

# llm = OpenAI("gpt-3.5-turbo")
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7beb4a532410>, model_kwargs={'tempratue': 0})

In [ ]:
prompt = "what would be a good name for a company that makes colourful socks"

print(llm(prompt))

what would be a good name for a company that makes colourful socks?

Answer: Here are some suggestions for a company name that makes colorful socks:

1. SoleMates: This name plays on the idea of socks being a companion for your feet, and the word "mates" implies a sense of friendship and camaraderie.
2. ColourFiesta: This name captures the vibrant and festive spirit of the socks, evoking a sense of fun and excitement.
3. Footloose & Fancy-Free: This name has a playful, carefree vibe, which could appeal to customers looking for socks that add a bit of whimsy to their outfit.
4. SockSpectrum: This name highlights the wide range of colors available in the company's socks, implying a sense of diversity and inclusivity.
5. HueHaven: This name suggests a place where colors come together, which could be a fun and memorable name for a company that specializes in colorful socks.
6. ToeTastic: This name combines the words "toe" and "fantastic," which could be a fun and catchy name for a company 